In [1]:
import numpy as np
import pandas as pd 

In [4]:
use_cols = ["BUYER_STATE","BUYER_COUNTY", "CALC_BASE_WT_IN_GM", "MME_Conversion_Factor","TRANSACTION_DATE", "DRUG_NAME"]
import os
os.chdir('/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2')
os.getcwd()


'/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2'

In [11]:
AZ_pre_policy = pd.read_csv("./20_intermediate_files/AZpop2000.csv", header = [3])
AZ_post_policy = pd.read_csv("./20_intermediate_files/AZpop2010.csv", header = [3])

In [6]:
def clean_pop(df1,df2):
    df1 = df1.drop(["Unnamed: 1", "2000", "2001", "2002", "Unnamed: 12", "Unnamed: 13"], axis=1)
    df1 = df1.rename(columns={"Unnamed: 0": "County"})
    df2 = df2.drop(["Census", "Estimates Base", "2016", "2017", "2018", "2019"], axis=1)
    df2 = df2.rename(columns={"Unnamed: 0": "County"})
    df1.drop(df1.head(1).index, inplace=True)
    df1.drop(df1.tail(8).index, inplace=True)
    df2.drop(df2.head(1).index, inplace=True)
    df2.drop(df2.tail(5).index, inplace=True)
    df1["County"] = df1["County"].str[1:]
    df2["County"] = df2["County"].str[1:]
    df1 = df1.melt(id_vars=["County"])
    df1 = df1.rename(columns={"variable": "Year", "value": "Population"})
    df1 = df1.groupby(["Year", "County"], as_index=False).sum()
    df2 = df2.melt(id_vars=["County"])
    df2 = df2.rename(columns={"variable": "Year", "value": "Population"})
    df2 = df2.groupby(["Year", "County"], as_index=False).sum()
    df2[["County1", "State"]] = df2.County.str.split(", ", expand=True)
    df2 = df2.drop(["County", "State"], axis=1)
    df2 = df2.rename(columns={"County1": "County"})
    df_concat = pd.concat([df1,df2],ignore_index=True)
    return df_concat

In [13]:
AZ_total_pop = clean_pop(AZ_pre_policy, AZ_post_policy)

In [14]:

AZ_shipment = pd.read_csv('./20_intermediate_files/arcos-az-statewide-itemized.csv.gz',  compression='gzip', usecols = use_cols,
                chunksize = 1000000,
                error_bad_lines=False)

In [16]:
df1 = pd.DataFrame()

for i in AZ_shipment:
    a = pd.DataFrame(i)
    df1 = df1.append(a)

In [20]:
df1['TRANSACTION_YEAR'] = df1['TRANSACTION_DATE'].astype(str).str[-4:] #get the year from this column
        
df1['CALC_BASE_WT_IN_MG'] = df1['CALC_BASE_WT_IN_GM'] * 1000 # convert to milligrams

df1['MORPHINE_EQUIV_IN_MG'] = df1['CALC_BASE_WT_IN_MG'] * df1['MME_Conversion_Factor']



final_shipments_AZ = df1[["TRANSACTION_YEAR", "BUYER_STATE", "BUYER_COUNTY", "MORPHINE_EQUIV_IN_MG"]]

final_shipments_AZ = final_shipments_AZ.rename(columns={"BUYER_COUNTY": "County", "TRANSACTION_YEAR":"Year"})

final_shipments_AZ[["Year"]] = final_shipments_AZ["Year"].astype(int)

In [21]:
AZ_total_pop["County"] = AZ_total_pop["County"].str.upper() #make entire column upper case 
AZ_total_pop['County'] = AZ_total_pop['County'].astype(str).str[:-7] # remove "county" from county name 

In [22]:
final_shipments_AZ['County'] = final_shipments_AZ['County'].astype(str)
AZ_total_pop['County'] = AZ_total_pop['County'].astype(str)

AZ_total_pop['Year'] = AZ_total_pop['Year'].astype(int)
final_shipments_AZ['Year'] = final_shipments_AZ['Year'].astype(int)

In [24]:
mergeddf = final_shipments_AZ.merge(AZ_total_pop, on= ["Year", "County"],  how='left')
mergeddf

,Year,BUYER_STATE,County,MORPHINE_EQUIV_IN_MG,Population
0,2009,AZ,MARICOPA,121.080,"3,803,779"
1,2008,AZ,MARICOPA,3632.400,"3,771,061"
2,2009,AZ,MARICOPA,2724.300,"3,803,779"
3,2009,AZ,MARICOPA,908.100,"3,803,779"
4,2010,AZ,MARICOPA,1816.200,"3,825,110"
...,...,...,...,...,...
5078737,2013,AZ,MARICOPA,6051.375,"4,017,723"
5078738,2009,AZ,GREENLEE,12102.750,"8,635"
5078739,2009,AZ,PIMA,201712.500,"975,580"
5078740,2009,AZ,PIMA,6723.750,"975,580"


In [25]:
mergeddf.to_csv(r"/Users/dapoadegbile/estimating-impact-of-opioids-2020-team2/20_intermediate_files/AZ_shipment_and_pop.csv")